
# AI_Pred_Rundeck — Pipeline Completo (Notebook)

Este notebook executa, **passo a passo**, o pipeline de previsão e detecção de anomalias:
1. Preparação do ambiente e pastas do projeto  
2. (Opcional) Instalação de dependências  
3. Geração/carregamento dos dados (`data/dados.csv`)  
4. ETL (limpeza e organização dos dados)  
5. Features (colunas auxiliares)  
6. Modelagem e Previsões (**Prophet** e **ARIMA**)  
7. Detecção de anomalias  
8. Visualização dos resultados  

> **Dica:** Rode célula por célula, de cima para baixo.



## 1. Dependências (execute se necessário)
Se você estiver em um ambiente **limpo** (por exemplo, nova máquina/venv), rode a célula abaixo para instalar as bibliotecas.
Se já estiver tudo instalado, pode pular.


In [ ]:

# Execute esta célula SOMENTE se precisar instalar as dependências:
# (Requer internet)
# %pip install -U pip pandas numpy pmdarima prophet "holidays[BR]" matplotlib



## 2. Estrutura de Pastas
Cria as pastas padrão do projeto:
- `data/` para dados de entrada
- `outputs/` para resultados
- `src/` para os scripts Python usados no pipeline


In [ ]:

from pathlib import Path

DATA_DIR = Path("data")
OUT_DIR = Path("outputs")
SRC_DIR = Path("src")

for p in (DATA_DIR, OUT_DIR, SRC_DIR):
    p.mkdir(parents=True, exist_ok=True)

print("Pastas criadas/ok:", DATA_DIR, OUT_DIR, SRC_DIR)



## 3. Código-Fonte do Pipeline
A célula abaixo grava os arquivos Python dentro de `src/`.
Se você já possui esses arquivos e quer mantê-los como estão, **pule esta célula**.


In [ ]:

from pathlib import Path, PurePosixPath

(SRC_DIR / "etl.py").write_text("#!/usr/bin/env python3\n\"\"\"\nETL b\u00e1sico para s\u00e9ries temporais.\n- L\u00ea data/dados.csv (ou caminho informado)\n- Normaliza colunas, ordena e remove duplicidades\n- Ajusta frequ\u00eancia (ex.: D/H), interpola e trata outliers (MAD clipping opcional)\n- Salva em outputs/clean.csv\n\"\"\"\nimport argparse\nimport numpy as np\nimport pandas as pd\nfrom pathlib import Path\n\ndef mad_clip(s: pd.Series, k: float = 3.5) -> pd.Series:\n    med = s.median()\n    mad = np.median(np.abs(s - med))\n    if mad == 0 or np.isnan(mad):\n        return s\n    lower = med - k * 1.4826 * mad\n    upper = med + k * 1.4826 * mad\n    return s.clip(lower, upper)\n\ndef main():\n    ap = argparse.ArgumentParser()\n    ap.add_argument(\"--input\", default=\"data/dados.csv\")\n    ap.add_argument(\"--output\", default=\"outputs/clean.csv\")\n    ap.add_argument(\"--freq\", default=\"D\", help=\"Frequ\u00eancia destino (ex.: D, H)\")\n    ap.add_argument(\"--cap_outliers\", action=\"store_true\", help=\"Ativa MAD clipping\")\n    args = ap.parse_args()\n\n    inp = Path(args.input)\n    outp = Path(args.output)\n    outp.parent.mkdir(parents=True, exist_ok=True)\n\n    df = pd.read_csv(inp, parse_dates=[\"timestamp\"])\n    # Normaliza nomes\n    cols = {c.lower(): c for c in df.columns}\n    if \"ds\" in df.columns and \"y\" in df.columns:\n        df = df.rename(columns={\"ds\": \"timestamp\", \"y\": \"value\"})\n    elif \"timestamp\" not in cols or \"value\" not in cols:\n        raise ValueError(\"Esperado colunas 'timestamp' e 'value' ou 'ds' e 'y'.\")\n\n    df = df[[\"timestamp\", \"value\"]].dropna().sort_values(\"timestamp\")\n    df = df.drop_duplicates(subset=[\"timestamp\"])\n\n    # Resample para frequ\u00eancia alvo\n    s = df.set_index(\"timestamp\")[\"value\"].asfreq(args.freq)\n    # Interpola e preenche bordas\n    s = s.interpolate(\"time\").ffill().bfill()\n\n    if args.cap_outliers:\n        s = mad_clip(s)\n\n    clean = s.reset_index().rename(columns={\"index\": \"timestamp\"})\n    clean.to_csv(outp, index=False)\n    print(f\"\u2705 Salvo: {outp} ({len(clean)} linhas)\")\n\nif __name__ == \"__main__\":\n    main()\n", encoding="utf-8")
(SRC_DIR / "features.py").write_text("#!/usr/bin/env python3\n\"\"\"\nGera\u00e7\u00e3o de features para modelos:\n- Lags: 1, 7, 28\n- Janelas m\u00f3veis: m\u00e9dia/std 7 dias\n- Sazonalidade c\u00edclica: dia da semana e dia do ano (seno/cosseno)\n- Flags de fim de semana e (opcional) feriado BR\nSalva em outputs/features.csv\n\"\"\"\nimport argparse\nimport numpy as np\nimport pandas as pd\nfrom pathlib import Path\n\ndef try_holidays_br(dates: pd.Series) -> pd.Series:\n    try:\n        import holidays\n        br = holidays.Brazil()\n        return dates.dt.date.map(lambda d: 1 if d in br else 0).astype(int)\n    except Exception:\n        return pd.Series(0, index=dates.index, dtype=int)\n\ndef main():\n    ap = argparse.ArgumentParser()\n    ap.add_argument(\"--input\", default=\"outputs/clean.csv\")\n    ap.add_argument(\"--output\", default=\"outputs/features.csv\")\n    ap.add_argument(\"--lags\", default=\"1,7,28\")\n    ap.add_argument(\"--roll\", type=int, default=7, help=\"Janela rolling (dias)\")\n    args = ap.parse_args()\n\n    path_in = Path(args.input)\n    path_out = Path(args.output)\n    path_out.parent.mkdir(parents=True, exist_ok=True)\n\n    df = pd.read_csv(path_in, parse_dates=[\"timestamp\"]).sort_values(\"timestamp\")\n    df = df.rename(columns={\"timestamp\": \"ds\", \"value\": \"y\"})\n\n    # Calend\u00e1rio/Sazonalidade\n    df[\"dow\"] = df[\"ds\"].dt.dayofweek\n    df[\"is_weekend\"] = (df[\"dow\"] >= 5).astype(int)\n    df[\"month\"] = df[\"ds\"].dt.month\n    doy = df[\"ds\"].dt.dayofyear\n\n    df[\"dow_sin\"] = np.sin(2 * np.pi * df[\"dow\"] / 7.0)\n    df[\"dow_cos\"] = np.cos(2 * np.pi * df[\"dow\"] / 7.0)\n    df[\"yoy_sin\"] = np.sin(2 * np.pi * doy / 365.25)\n    df[\"yoy_cos\"] = np.cos(2 * np.pi * doy / 365.25)\n\n    # Feriados BR (opcional)\n    df[\"is_holiday_br\"] = try_holidays_br(df[\"ds\"])\n\n    # Lags\n    lags = [int(x) for x in args.lags.split(\",\") if x.strip()]\n    for L in lags:\n        df[f\"lag_{L}\"] = df[\"y\"].shift(L)\n\n    # Rolling\n    roll = args.roll\n    df[f\"roll_mean_{roll}\"] = df[\"y\"].rolling(roll, min_periods=1).mean()\n    df[f\"roll_std_{roll}\"]  = df[\"y\"].rolling(roll, min_periods=1).std().fillna(0)\n\n    # Remove linhas iniciais com NaN em lags\n    df = df.dropna().reset_index(drop=True)\n\n    df.to_csv(path_out, index=False)\n    print(f\"\u2705 Salvo: {path_out} ({len(df)} linhas)\")\n\nif __name__ == \"__main__\":\n    main()\n", encoding="utf-8")
(SRC_DIR / "train_prophet.py").write_text("#!/usr/bin/env python3\n\"\"\"\nTreino/forecast com Prophet:\n- L\u00ea outputs/clean.csv (ds,y)\n- Adiciona feriados BR\n- Gera previs\u00e3o para horizonte (--horizon) e salva CSV\n\"\"\"\nimport argparse\nimport pandas as pd\nfrom pathlib import Path\n\ndef main():\n    ap = argparse.ArgumentParser()\n    ap.add_argument(\"--input\", default=\"outputs/clean.csv\")\n    ap.add_argument(\"--output\", default=\"outputs/prophet_forecast.csv\")\n    ap.add_argument(\"--horizon\", type=int, default=30, help=\"Per\u00edodos futuros (ex.: dias)\")\n    ap.add_argument(\"--freq\", default=\"D\", help=\"Frequ\u00eancia (D/H)\")\n    args = ap.parse_args()\n\n    inp = Path(args.input)\n    outp = Path(args.output)\n    outp.parent.mkdir(parents=True, exist_ok=True)\n\n    df = pd.read_csv(inp, parse_dates=[\"timestamp\"]).sort_values(\"timestamp\")\n    df = df.rename(columns={\"timestamp\": \"ds\", \"value\": \"y\"})\n\n    from prophet import Prophet\n    m = Prophet(\n        yearly_seasonality=True,\n        weekly_seasonality=True,\n        daily_seasonality=False,\n        changepoint_prior_scale=0.5,\n        interval_width=0.95\n    )\n    # Feriados BR\n    m.add_country_holidays(country_name='BR')\n\n    m.fit(df)\n\n    future = m.make_future_dataframe(periods=args.horizon, freq=args.freq)\n    fcst = m.predict(future)\n\n    # Salva somente colunas \u00fateis\n    cols = [\"ds\",\"yhat\",\"yhat_lower\",\"yhat_upper\"]\n    fcst[cols].to_csv(outp, index=False)\n    print(f\"\u2705 Forecast salvo em {outp} ({len(fcst)} linhas)\")\n\nif __name__ == \"__main__\":\n    main()\n", encoding="utf-8")
(SRC_DIR / "train_arima.py").write_text("#!/usr/bin/env python3\n\"\"\"\nTreino/forecast com Auto-ARIMA (pmdarima):\n- L\u00ea outputs/clean.csv\n- Detecta sazonalidade semanal (m=7) para dados di\u00e1rios; para hor\u00e1rio usa m=24\n- Salva previs\u00e3o com intervalo de confian\u00e7a\n\"\"\"\nimport argparse\nimport pandas as pd\nfrom pathlib import Path\nimport pmdarima as pm\n\ndef main():\n    ap = argparse.ArgumentParser()\n    ap.add_argument(\"--input\", default=\"outputs/clean.csv\")\n    ap.add_argument(\"--output\", default=\"outputs/arima_forecast.csv\")\n    ap.add_argument(\"--horizon\", type=int, default=30)\n    ap.add_argument(\"--freq\", default=\"D\", help=\"D ou H\")\n    args = ap.parse_args()\n\n    inp = Path(args.input)\n    outp = Path(args.output)\n    outp.parent.mkdir(parents=True, exist_ok=True)\n\n    df = pd.read_csv(inp, parse_dates=[\"timestamp\"]).sort_values(\"timestamp\")\n    s = df.set_index(\"timestamp\")[\"value\"].asfreq(args.freq)\n\n    m = 7 if args.freq.upper() == \"D\" else 24\n    model = pm.auto_arima(\n        s, seasonal=True, m=m,\n        stepwise=True, suppress_warnings=True,\n        error_action=\"ignore\", information_criterion=\"aic\"\n    )\n\n    yhat, conf = model.predict(n_periods=args.horizon, return_conf_int=True, alpha=0.05)\n    idx = pd.date_range(s.index[-1] + pd.tseries.frequencies.to_offset(args.freq),\n                        periods=args.horizon, freq=args.freq)\n\n    out = pd.DataFrame({\n        \"ds\": idx,\n        \"yhat\": yhat,\n        \"yhat_lower\": conf[:,0],\n        \"yhat_upper\": conf[:,1],\n    })\n    out.to_csv(outp, index=False)\n    print(f\"\u2705 Forecast salvo em {outp} ({len(out)} linhas)\")\n\nif __name__ == \"__main__\":\n    main()\n", encoding="utf-8")
(SRC_DIR / "anomalies.py").write_text("#!/usr/bin/env python3\n\"\"\"\nDetec\u00e7\u00e3o de anomalias:\n- Junta hist\u00f3rico (clean.csv) com forecast (Prophet/ARIMA)\n- Marca anomalia se y estiver fora de [yhat_lower, yhat_upper]\n- Se limites n\u00e3o existirem, usa z-score rolling (janela=30)\n\"\"\"\nimport argparse\nimport pandas as pd\nimport numpy as np\nfrom pathlib import Path\n\ndef detect_by_pi(obs: pd.DataFrame) -> pd.DataFrame:\n    cond_pi = (obs[\"y\"] < obs[\"yhat_lower\"]) | (obs[\"y\"] > obs[\"yhat_upper\"])\n    obs[\"anomaly\"] = cond_pi.astype(int)\n    obs[\"residual\"] = obs[\"y\"] - obs[\"yhat\"]\n    return obs\n\ndef detect_by_zscore(obs: pd.DataFrame, window: int = 30, z: float = 3.0) -> pd.DataFrame:\n    s = obs[\"y\"]\n    mu = s.rolling(window, min_periods=1).mean()\n    sd = s.rolling(window, min_periods=1).std().fillna(0.0)\n    zscore = (s - mu) / (sd.replace(0, np.nan))\n    obs[\"yhat\"] = mu\n    obs[\"yhat_lower\"] = mu - z * sd\n    obs[\"yhat_upper\"] = mu + z * sd\n    obs[\"residual\"] = s - mu\n    obs[\"anomaly\"] = (zscore.abs() > z).astype(int)\n    return obs\n\ndef main():\n    ap = argparse.ArgumentParser()\n    ap.add_argument(\"--history\", default=\"outputs/clean.csv\")\n    ap.add_argument(\"--forecast\", required=True, help=\"CSV de forecast (prophet/arima)\")\n    ap.add_argument(\"--output\", default=\"outputs/anomalies.csv\")\n    args = ap.parse_args()\n\n    hist = pd.read_csv(args.history, parse_dates=[\"timestamp\"])\n    hist = hist.rename(columns={\"timestamp\": \"ds\", \"value\": \"y\"})\n\n    fcst = pd.read_csv(args.forecast, parse_dates=[\"ds\"])\n\n    # Usa apenas per\u00edodo observado (interse\u00e7\u00e3o) para checagem de anomalias em hist\u00f3rico\n    obs = hist.merge(fcst, on=\"ds\", how=\"left\")\n\n    if {\"yhat_lower\",\"yhat_upper\"}.issubset(obs.columns):\n        result = detect_by_pi(obs)\n    else:\n        result = detect_by_zscore(obs)\n\n    outp = Path(args.output)\n    outp.parent.mkdir(parents=True, exist_ok=True)\n    result.to_csv(outp, index=False)\n    print(f\"\u2705 Anomalias salvas em {outp} (total={result['anomaly'].sum()} marcadas)\")\n\nif __name__ == \"__main__\":\n    main()\n", encoding="utf-8")

print("Arquivos gravados em", SRC_DIR)
list(SRC_DIR.iterdir())



## 4. Dados de Exemplo (gera se não existir)
Se você já possui `data/dados.csv`, **pule**.  
Se não possuir, rode a célula abaixo para **gerar um dataset sintético** com tendência, sazonalidades e algumas anomalias.


In [ ]:

import numpy as np
import pandas as pd

data_file = DATA_DIR / "dados.csv"
if not data_file.exists():
    rng = pd.date_range("2024-01-01", "2025-08-31", freq="D")
    n = len(rng)
    rs = np.random.RandomState(42)

    trend = np.linspace(100, 140, n)
    weekly = 10 * np.sin(2 * np.pi * (rng.dayofweek.values / 7.0))
    yearly = 15 * np.sin(2 * np.pi * (rng.dayofyear.values / 365.25))
    noise = rs.normal(0, 5, n)

    value = trend + weekly + yearly + noise
    anom_idx = rs.choice(n, size=10, replace=False)
    value[anom_idx] += rs.choice([-40, 40], size=10)
    value = np.maximum(value, 0)

    df = pd.DataFrame({"timestamp": rng, "value": value.round(2)})
    df.to_csv(data_file, index=False)
    print(f"Gerei {data_file} com {len(df)} linhas.")
else:
    print(f"Arquivo já existe: {data_file}")



## 5. ETL — Limpeza e Organização
Lê `data/dados.csv`, ordena por data, preenche falhas e pode limitar outliers (**MAD clipping**).  
Saída: `outputs/clean.csv`.


In [ ]:

!python src/etl.py --input data/dados.csv --output outputs/clean.csv --freq D --cap_outliers



## 6. Features — Colunas Auxiliares
Cria lags (1, 7, 28), médias móveis, indicadores cíclicos (dia da semana/ano) e flag de feriado BR.  
Saída: `outputs/features.csv`.


In [ ]:

!python src/features.py --input outputs/clean.csv --output outputs/features.csv --lags "1,7,28" --roll 7



## 7. Previsão com Prophet
Usa sazonalidades e feriados do Brasil.  
Saída: `outputs/prophet_forecast.csv`.


In [ ]:

!python src/train_prophet.py --input outputs/clean.csv --output outputs/prophet_forecast.csv --horizon 30 --freq D



## 8. Previsão com ARIMA (Auto-ARIMA)
Detecta sazonalidade automaticamente (m=7 para diário, m=24 para horário).  
Saída: `outputs/arima_forecast.csv`.


In [ ]:

!python src/train_arima.py --input outputs/clean.csv --output outputs/arima_forecast.csv --horizon 30 --freq D



## 9. Detecção de Anomalias
Compara histórico com previsões; marca como anomalia quando o valor observado sai dos limites do intervalo previsto.  
Saídas: `outputs/anomalies_prophet.csv` e `outputs/anomalies_arima.csv`.


In [ ]:

!python src/anomalies.py --history outputs/clean.csv --forecast outputs/prophet_forecast.csv --output outputs/anomalies_prophet.csv
!python src/anomalies.py --history outputs/clean.csv --forecast outputs/arima_forecast.csv   --output outputs/anomalies_arima.csv



## 10. Visualização (Gráficos)
Gráficos simples com **matplotlib**:  
- Série histórica vs. previsão (Prophet)  
- Destaque dos pontos marcados como anomalia  


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

hist = pd.read_csv("outputs/clean.csv", parse_dates=["timestamp"]).sort_values("timestamp")
fcst = pd.read_csv("outputs/prophet_forecast.csv", parse_dates=["ds"]).sort_values("ds")
anom = pd.read_csv("outputs/anomalies_prophet.csv", parse_dates=["ds"]).sort_values("ds")

# Merge para período observado
obs = hist.rename(columns={"timestamp":"ds","value":"y"}).merge(
    fcst[["ds","yhat","yhat_lower","yhat_upper"]],
    on="ds", how="left"
)

# Plot histórico + previsão + intervalos
plt.figure()
plt.plot(obs["ds"], obs["y"], label="Histórico")
plt.plot(fcst["ds"], fcst["yhat"], label="Previsão (Prophet)")
plt.fill_between(fcst["ds"], fcst["yhat_lower"], fcst["yhat_upper"], alpha=0.2, label="Intervalo")
plt.legend()
plt.title("Histórico vs. Previsão (Prophet)")
plt.xlabel("Data")
plt.ylabel("Valor")
plt.show()

# Plot anomalias (marcador 'x'). Sem cores explícitas.
anom_points = anom[anom["anomaly"] == 1]
plt.figure()
plt.plot(obs["ds"], obs["y"], label="Histórico")
if not anom_points.empty:
    plt.scatter(anom_points["ds"], anom_points["y"], label="Anomalias (Prophet)", marker="x")
plt.legend()
plt.title("Anomalias Detectadas (Prophet)")
plt.xlabel("Data")
plt.ylabel("Valor")
plt.show()

# Pré-visualização de alguns arquivos
for f in ["outputs/features.csv", "outputs/prophet_forecast.csv", "outputs/arima_forecast.csv",
          "outputs/anomalies_prophet.csv", "outputs/anomalies_arima.csv"]:
    try:
        df = pd.read_csv(f)
        print(f"\nPrévia de {f}:")
        display(df.head())
    except Exception as e:
        print(f"Falha ao ler {f}: {e}")
